In [ ]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]


In [ ]:
# 1
%pip install -q requests

# 2
import requests

# 3
def get_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# 4
rfc_url = 'https://www.rfc-editor.org/rfc/rfc2549.html'
print(get_text_from_url(rfc_url))

In [ ]:
# 1
%pip install -q openai==1.42.0
%pip install -q pydantic

# 2
from pprint import pp
from openai import OpenAI
from pydantic import BaseModel

# 3
client = OpenAI()

# 4
class StructuredSummary(BaseModel):
    facts: list[str]
    analysis: list[str]
    recommendations: list[str]

# 5
messages = [
    {
        "role": "system", 
        "content": "Extract a structured summary of the text. The summary must be less than a few words. You should respond to follow up questions."
    },
]

# 6
def get_structured_summary(input):
    messages.append({"role": "user", "content": input})

    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format=StructuredSummary
    ) 

    return response.choices[0].message.parsed

# 7
pigeon_rfc = get_text_from_url(rfc_url)
summary = get_structured_summary(pigeon_rfc)

# 8
import json
print(json.dumps(summary.model_dump(), indent=4))


In [ ]:
# 1
messages.append({"role": "assistant", "content": summary.model_dump_json()})

# 2
while True:
    # 3
    user_input = input("Please enter your input: ")
 
    # 4
    if user_input.lower() == 'exit':
        break

    # 5
    summary = get_structured_summary(user_input)
    print(json.dumps(summary.model_dump(), indent=4))

    messages.append({"role": "assistant", "content": summary.model_dump_json()})
